In [2]:
import tensorflow as tf # Tensorflow 2.3
import tensorflow_hub as hub # Tensorflow-hub 0.12
import PIL.Image as Image # Pillow 
import numpy as np # Numpy
from tqdm import tqdm # progress bar package
import tensorflow_datasets as tfds
import os
from time import process_time
from memory_profiler import memory_usage
from ipynb.fs.full.utilities import *
import tempfile
import tracemalloc

#se sxesi me ton xrono: default_timer == perf_counter
#perf_counter:does include time elapsed during sleep
#process_time(): does NOT include time elapsed during sleep

c:\Users\Katerina\Anaconda3\envs\benchmarkapp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
'''
task = int(input("Choose a problem domain by typing the corresponding number: \n1.Image\n2.Text\n3.Video\n4.Audio"))
if(task == 1):
    task = int(input("Choose a task by typing the corresponding number:\n1.Image Classification\n2.Image Segmentation\n3.Pose Estimation"))
    if(task == 1):
        task = int(input("Choose a model by typing the corresponding number:\n1.MobileNet V2\n2.ResNet50 V1\n3.Inception V3\n4.NASNet-A large\n5.EfficientNet V2 M"))

dataset = input('Absolute path to the dataset')
values = input('Absolute path to the text file of the groundtruth')
'''

batch = int(input("Insert batch size"))
if batch == 1:
    batch = None

## Image Classification

In [4]:
#Groundtruth
val_groundtruth = load_values('Datasets/ImageNet/val.txt')

### MobileNetV2

In [5]:
# Load Model
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", output_shape=[1001])
])
model.build([batch,224,224,3]) # Batch input shape
image_shape = (224,224)

In [8]:
# Inference and metrics

t1 = 0 # Variable for correct predictions
t5 = 0 # Variable for top-5 correct predictions

imagenum = 0
duration = 0
data = np.zeros((1000 , 224, 224, 3))

for file in os.listdir('Datasets/ImageNet')[:1000]:
    image = Image.open(os.path.join('Datasets/ImageNet', file), 'r')
    data[imagenum,:] = preprocessing(image, image_shape)[0] # Preprocessing
    imagenum += 1

tracemalloc.start()
start = process_time()    
prediction = model.predict(data,batch_size = batch)
duration += (process_time()-start)

for i in range(imagenum):
    if((val_groundtruth[i]+1) in np.argpartition(prediction[i], -5)[-5:]): # argpartition σε Ο(n) για τα top5
            t5+=1
            if(val_groundtruth[i]+1) == np.argmax(prediction[i]):
                t1+=1

print('Size: 14MB')
print('Peak memory usage is: {} KiB '.format(round(((tracemalloc.get_traced_memory()[1])/1024))))
print('Top-1 accuracy for {} images is: {} %'.format(imagenum, round((t1/(imagenum))*100,1))) # Printing Accuracy.
print('Top-5 accuracy for {} images is: {} %'.format(imagenum, round((t5/(imagenum))*100,1))) # Printing Accuracy.
print('Latency is: {} s'.format(round(duration/(imagenum),2))) # Printing Latency.
print('Throughput is: {} samples/sec'.format(round((imagenum)/duration,2))) # Printing Throughput.
tracemalloc.stop()


Size: 14MB
Peak memory usage is: 592070 KiB 
Top-1 accuracy for 1000 images is: 70.1 %
Top-5 accuracy for 1000 images is: 90.2 %
Latency is: 0.54 s
Throughput is: 1.84 samples/sec


### ResNet50 V1

In [9]:
#Load the model

model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/tensorflow/resnet_50/classification/1") # First input is the directory to the saved model.
])
model.build([batch, 224, 224, 3]) # Batch input shape.

In [10]:
# Inference and metrics
t1 = 0 # Variable for correct predictions
t5 = 0 # Variable for top-5 correct predictions

imagenum = 0
data = np.zeros((1000 , 224, 224, 3))

for file in os.listdir('Datasets/ImageNet')[:1000]:
    image = Image.open(os.path.join('Datasets/ImageNet', file), 'r')
    data[imagenum,:] = preprocessing(image, image_shape)[0] # Preprocessing
    imagenum += 1

tracemalloc.start()
start = process_time()
prediction = model.predict(data,batch_size = batch)
duration += (process_time()-start)

for i in range(imagenum):
    if((val_groundtruth[i]) in np.argpartition(prediction[i], -5)[-5:]): # argpartition σε Ο(n) για τα top5
        t5+=1
        if(val_groundtruth[i]) == np.argmax(prediction[i]):
            t1+=1

print('Size: 98MB')
print('Peak memory usage is: {} KiB'.format(round(((tracemalloc.get_traced_memory()[1])/1024))))
print('Top-1 accuracy for {} images is: {} %'.format(imagenum, round((t1/(imagenum+1))*100,1))) # Printing Accuracy.
print('Top-5 accuracy for {} images is: {} %'.format(imagenum, round((t5/(imagenum+1))*100,1))) # Printing Accuracy.
print('Latency is: {} s'.format(round(duration/(imagenum),2))) # Printing Latency.
print('Throughput is: {} samples/sec'.format(round((imagenum)/duration,2))) # Printing Throughput.
tracemalloc.stop()

Size: 98MB
Peak memory usage is: 593280 KiB
Top-1 accuracy for 1000 images is: 74.2 %
Top-5 accuracy for 1000 images is: 91.6 %
Latency is: 3.42 s
Throughput is: 0.29 samples/sec


### Inception V3

In [11]:
#Load the model

model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/classification/5") # First input is the directory to the saved model.
])
model.build([batch, 299, 299, 3]) # Batch input shape.

In [12]:
# Inference and metrics
t1 = 0 # Variable for correct predictions
t5 = 0 # Variable for top-5 correct predictions

imagenum = 0
duration = 0
data = np.zeros((1000 , 224, 224, 3))

for file in os.listdir('Datasets/ImageNet')[:1000]:
    image = Image.open(os.path.join('Datasets/ImageNet', file), 'r')
    data[imagenum,:] = preprocessing(image, image_shape)[0] # Preprocessing
    imagenum += 1

tracemalloc.start()   
start = process_time()    
prediction = model.predict(data,batch_size = batch)
duration += (process_time()-start)

for i in range(imagenum):
    if((val_groundtruth[i]+1) in np.argpartition(prediction[i], -5)[-5:]): # argpartition σε Ο(n) για τα top5
        t5+=1
        if(val_groundtruth[i]+1) == np.argmax(prediction[i]):
            t1+=1

print('Size: 92MB')
print('Peak memory usage is: {} KiB'.format(round(((tracemalloc.get_traced_memory()[1])/1024))))
print('Top-1 accuracy for {} images is: {} %'.format(imagenum, round((t1/(imagenum))*100,1))) # Printing Accuracy.
print('Top-5 accuracy for {} images is: {} %'.format(imagenum, round((t5/(imagenum))*100,1))) # Printing Accuracy.
print('Latency is: {} s'.format(round(duration/(imagenum),2))) # Printing Latency.
print('Throughput is: {} samples/sec'.format(round((imagenum)/duration,2))) # Printing Throughput.
tracemalloc.stop()

Size: 92MB
Peak memory usage is: 594512 KiB
Top-1 accuracy for 1000 images is: 69.2 %
Top-5 accuracy for 1000 images is: 87.5 %
Latency is: 2.21 s
Throughput is: 0.45 samples/sec


### NASNet-A large

In [13]:
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/nasnet_large/classification/5")
])
model.build([batch, 331, 331, 3])  # Batch input shape.
image_shape = (331,331)

In [14]:
# Inference and metrics
t1 = 0 # Variable for correct predictions
t5 = 0 # Variable for top-5 correct predictions

imagenum = 0
duration = 0
data = np.zeros((100 , 331, 331, 3))

for file in os.listdir('Datasets/ImageNet')[:100]:
    image = Image.open(os.path.join('Datasets/ImageNet', file), 'r')
    data[imagenum,:] = preprocessing(image, image_shape)[0] # Preprocessing
    imagenum += 1

tracemalloc.start()   
start = process_time()    
prediction = model.predict(data,batch_size = batch)
duration += (process_time()-start)

for i in tqdm(range(imagenum)):
    if((val_groundtruth[i]+1) in np.argpartition(prediction[i], -5)[-5:]): # argpartition σε Ο(n) για τα top5
            t5+=1
            if(val_groundtruth[i]+1) == np.argmax(prediction[i]):
                t1+=1

print('Size: 343MB')
print('Peak memory usage is: {} KiB'.format(round(((tracemalloc.get_traced_memory()[1])/1024))))
print('Top-1 accuracy for {} images is: {} %'.format(imagenum, round((t1/(imagenum))*100,1))) # Printing Accuracy.
print('Top-5 accuracy for {} images is: {} %'.format(imagenum, round((t5/(imagenum))*100,1))) # Printing Accuracy.
print('Latency is: {} s'.format(round(duration/(imagenum),2))) # Printing Latency.
print('Throughput is: {} samples/sec'.format(round((imagenum)/duration,2))) # Printing Throughput.
tracemalloc.stop()

100%|██████████| 100/100 [00:00<00:00, 547.45it/s]


Size: 343MB
Peak memory usage is: 135620 KiB
Top-1 accuracy for 100 images is: 79.0 %
Top-5 accuracy for 100 images is: 94.0 %
Latency is: 21.82 s
Throughput is: 0.05 samples/sec


### EfficientNetV2M

In [15]:
#Load Model
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_m/classification/2")
])
model.build([batch, 480, 480, 3])  # Batch input shape.
image_shape = (480,480)

In [17]:
# Inference and metrics
t1 = 0 # Variable for correct predictions
t5 = 0 # Variable for top-5 correct predictions

imagenum = 0
duration = 0
data = np.zeros((100 , 480, 480, 3))

for file in os.listdir('Datasets/ImageNet')[:100]:
    image = Image.open(os.path.join('Datasets/ImageNet', file), 'r')
    data[imagenum,:] = preprocessing(image, image_shape)[0] # Preprocessing
    imagenum += 1

tracemalloc.start()   
start = process_time()    
prediction = model.predict(data,batch_size = batch)
duration += (process_time()-start)

for i in tqdm(range(imagenum)):
    if((val_groundtruth[i]) in np.argpartition(prediction[i], -5)[-5:]): # argpartition σε Ο(n) για τα top5
            t5+=1
            if(val_groundtruth[i]) == np.argmax(prediction[i]):
                t1+=1

print('Size: 220MB')
print('Peak memory usage is: {} KiB'.format(round(((tracemalloc.get_traced_memory()[1])/1024))))
print('Top-1 accuracy for {} images is: {} %'.format(imagenum, round((t1/(imagenum))*100,1))) # Printing Accuracy.
print('Top-5 accuracy for {} images is: {} %'.format(imagenum, round((t5/(imagenum))*100,1))) # Printing Accuracy.
print('Latency is: {} s'.format(round(duration/(imagenum),2))) # Printing Latency.
print('Throughput is: {} samples/sec'.format(round((imagenum)/duration,2))) # Printing Throughput.
tracemalloc.stop()

100%|██████████| 100/100 [00:00<00:00, 4833.26it/s]

Size: 220MB
Peak memory usage is: 3515714 KiB
Top-1 accuracy for 100 images is: 84.0 %
Top-5 accuracy for 100 images is: 97.0 %
Latency is: 23.45 s
Throughput is: 0.04 samples/sec
